In [1]:
import urllib 
import lxml.html
from bs4 import BeautifulSoup 
import time
import os
import constants
import pandas as pd
import psycopg2
import re

In [2]:
def product_info(product_list):
    """
    This function scrapes the Pet Smart website to isolate information
    into separate variables from the url passed to it and exports the 
    information to a pickle file. The pickle file can be imported as a 
    dataframe, cleaned up, then stored into a database.

    Input:
    A list of urls of pet food product pages generated from crawler.py

    Output:

    nutrition_df.pkl
    name(str), productID(int), price(str), food_type(str), life_stage(str), 
    health_consideration(str), flavor(str),primary_ingredient(str), url(str)
    package_weight(str), , ingredient_list(str) and a set of columns iterated from the 
    "Guaranteed Analysis" portion of the product page, which lists things like 
    %protein and % moisture."""

    nutrition_dict_list = []
    def nutrition_regex(tag_list, nutrition_dict):

        """This function goes to the Guaranteed Analysis portion of the page
           and isolates the percentage of each nutritional component listed
           into it's appropriate key.
           
           Input: a list of sections of the webpage to search and the 
           dictionary to store the information in that is defined below.
           
           Output: Key: Value pairs listing the percentage of each 
           nutritional component. Values are percentages. These get added
           to the dictionary of product information defined in the overall
           function."""

        for tag in tag_list:
            if "Guaranteed Analysis" in tag:
                new = tag.split("<br/>")  
                for i, line in enumerate(new):
                    if "Crude Protein (min" in line:
                        nutrition_dict['crude_protein_min']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Crude Protein (max" in line:
                        nutrition_dict['crude_protein_max']= \
                        re.findall("\)\s*(.*?)\s*%", line)
                    elif "Crude Fat (min" in line:
                        nutrition_dict['crude_fat_min']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Crude Fat (max" in line:
                        nutrition_dict['crude_fat_max']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Crude Fiber (min" in line:
                        nutrition_dict['crude_fiber_min']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Crude Fiber (max" in line:
                        nutrition_dict['crude_fiber_max']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Moisture (min" in line:
                        nutrition_dict['moisture_min']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Moisture (max" in line:
                        nutrition_dict['moisture_max']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Ash (min" in line:
                        nutrition_dict['ash_min']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Ash (max" in line:
                        nutrition_dict['ash_max']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Taurine (min" in line:
                        nutrition_dict['taurine_min']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    elif "Taurine (max" in line:
                        nutrition_dict['taurine_max']= \
                        re.findall("\)\s*(.*?)\s*%", line) 
                    else:                        
                        nutrition_dict['tag'] = None
            else:
                None
        return nutrition_dict

    #Open product pages from the product url list created from the crawler function      
    for url in product_list: 

        nutrition_dict = {}        
        try:
        # Open each url, turn each page into a Beautiful Soup object
            r = urllib.urlopen(url)
            soup_page = BeautifulSoup(r, 'lxml') #'html.parser'
        except:
            print "Error getting page" 

        #check HTTP status codes while running (200 is good)
        print urllib.urlopen(url).getcode()

        try:        
            name_box = soup_page.find('div', attrs={'class': 'product-name'})
            prodID_box = soup_page.find('span', attrs={'class': 'productID'})        
            nutrition_dict['name'] = name_box.get_text().strip()  
            nutrition_dict['product_id'] = prodID_box.get_text()
        except:
            pass   

        try:
            price_box = soup_page.find('span', attrs={'class': 'price-regular'})
            nutrition_dict['price'] = price_box.get_text()
        except:
            nutrition_dict['price']=''
        try:
            nutrition_dict['url']=url
        except:
            nutrition_dict['url']=''            

        #This section defines the places interesting variables could be.
        #The html is messy, so I will define all the "b" tags and 
        #iterate through them below.   

        type_box = soup_page.find('div', attrs={'class': 'tab-content'})

        try:
            zero = type_box.find_all('b')[0]
        except:
            zero =  ''             
        try:
            one = type_box.find_all('b')[1]
        except:
            one = ''
        try:
            two = type_box.find_all('b')[2]
        except:
            two = ''
        try:
            three = type_box.find_all('b')[3]
        except:
            three = ''
        try:
            four = type_box.find_all('b')[4]
        except:
            four = ''
        try:
            five = type_box.find_all('b')[5]
        except:
            five = ''                      
        try:
            six = type_box.find_all('b')[6]
        except:
            six = ''
        try:
            seven = type_box.find_all('b')[7]
        except:
            seven = ''
        try:
            eight = type_box.find_all('b')[8]
        except:
            eight = ''
        try:
            nine = type_box.find_all('b')[9]
        except:
            nine = ''

        #Make a list of the variables
        info_list = [zero, one, two, three, four, five, six, 
                     seven, eight, nine] 

        #Iterate through the list to find where the information is
        #and put it into its appropriate key.
        for info in info_list:           
            if "Ingredients:" in info:  
                nutrition_dict['ingredients'] = info.next_sibling
            elif 'Food Type:' in info:
                nutrition_dict['food_type'] = info.next_sibling.strip()  
            elif 'Life Stage:' in info:
                nutrition_dict['life_stage'] = info.next_sibling.strip() 
            elif 'Health Consideration:' in info:
                nutrition_dict['health_consideration'] = \
                info.next_sibling.strip() 
            elif 'Flavor:' in info:
                nutrition_dict['flavor'] = info.next_sibling.strip() 
            elif 'Primary Ingredient:' in info:
                nutrition_dict['primary_ingredient'] = info.next_sibling
            elif 'Package Weight:' in info:
                nutrition_dict['package_weight'] = info.next_sibling.strip()
            else:
                None

        #This section defines the places "Guaranteed Analysis" could be.
        #The html is messy, so I will look in several places, and parse
        #through it using the nutrition_regex function above (called below).                    
        try:
            tag0 = str(type_box.findAll('p')[0].next_sibling).strip()
        except:
            tag0 = '' 
        try:
            tag1 = str(type_box.findAll('p')[1].next_sibling).strip()
        except:
            tag1 = ''       
        try:
            tag2 = str(type_box.findAll('p')[2].next_sibling).strip()
        except:
            tag2 = ''     
        try:
            tag3 = str(type_box.findAll('p')[3].next_sibling).strip()
        except:
            tag3 = ''
        try:
            tag4 = str(type_box.findAll('p')[4].next_sibling).strip()
        except:
            tag4 = ''

        #Make a list of the variables
        tag_list = [tag0, tag1, tag2, tag3, tag4]               

        #This calls the nutritional information parser from above,
        #feeding in the list of places to look on the page (tags)
        #for the nutritional information
        nutrition_dict = nutrition_regex(tag_list, nutrition_dict)   

        #append all keys:values to nutrition dictionary list
        nutrition_dict_list.append(nutrition_dict)    

    #turn dictionary list into a dataframes    
    nutrition_df = pd.DataFrame(nutrition_dict_list) 

    #export the dataframe from the function as a pickle file to access later
    nutrition_df.to_pickle('product_info_df.pkl')

    return nutrition_dict_list

product_info(constants.test_product_list5) 

200
200
200
200
200


[{'ash_max': ['3.5'],
  'crude_fat_min': ['2'],
  'crude_fiber_max': ['1.5'],
  'crude_protein_min': ['13'],
  'flavor': u'COD, Sole & Shrimp',
  'food_type': u'Wet Food',
  'health_consideration': u'General Health',
  'ingredients': u' Cod, Liver, Meat by-Products, Fish, Fish Broth, Sole, Shrimp, \r\nArtificial and Natural Flavors, Guar Gum, Added Color (Red 3 and Other Color), Calcium \r\nPhosphate, Potassium Chloride, Salt, Zinc Sulfate, Thiamine Mononitrate, Vitamin E \r\nSupplement, Ferrous Sulfate, Niacin, Manganese Sulfate, Calcium Pantothenate, Vitamin A \r\nSupplement, Copper Sulfate, Menadione Sodium Bisulfite Complex (Source of Vitamin K \r\nActivity), Pyridoxine Hydrochloride, Riboflavin Supplement, Vitamin B-12 Supplement, \r\nBiotin, Folic Acid, Vitamin D-3 Supplement, Potassium Iodide. \r\n',
  'life_stage': u'Adult 1-10 yrs',
  'moisture_max': ['78'],
  'name': u'Fancy Feast\xae Classic Adult Cat Food',
  'package_weight': u'3oz',
  'price': u'N/A',
  'primary_ingredien

In [4]:
#import the pickle file as a dataframe:
#look in pet-products-scraper-data folder
products = pd.read_pickle('product_info_df.pkl')
len(products)
products.head()

,ash_max,crude_fat_min,crude_fiber_max,crude_protein_min,flavor,food_type,health_consideration,ingredients,life_stage,moisture_max,name,package_weight,price,primary_ingredient,product_id,tag,taurine_min,url
0,[3.5],[2],[1.5],[13],"COD, Sole & Shrimp",Wet Food,General Health,"Cod, Liver, Meat by-Products, Fish, Fish Brot...",Adult 1-10 yrs,[78],Fancy Feast® Classic Adult Cat Food,3oz,N/A,COD,1221059,None,[0.05],http://www.petsmart.com/cat/food-and-health/fo...
1,[3.5],[5],[1],[10],Mixed Grill,Wet Food,General Health,"Meat by-Products, Water Sufficient for Proces...",Adult 1-10 yrs,[78],Purina® Friskies® Classic Paté Cat Food,5.5oz,N/A,Liver,1221026,None,[0.05],http://www.petsmart.com/cat/food-and-health/fo...
2,[2.5],[2.5],[1.0],[9.0],NaN,NaN,NaN,NaN,NaN,[82.0],Purina® Friskies® Savory Shreds Cat Food,NaN,N/A,NaN,5121705,None,[0.05],http://www.petsmart.com/cat/food-and-health/fo...
3,[2.7],[2],[1.5],[12],Seafood Feast,Wet Food,General Health,"Fish Broth, Ocean Fish, Liver, Wheat Gluten, ...",Adult 1-10 yrs,[78],Fancy Feast® Grilled Adult Cat Food,3oz,N/A,Chicken Broth,5121709,None,[0.05],http://www.petsmart.com/cat/food-and-health/fo...
4,NaN,NaN,NaN,NaN,Turkey,Wet Food,General Health,"Poultry Broth, Turkey, Wheat Gluten, Liver, M...",Adult 1-10 yrs,NaN,Fancy Feast® Gravy Lovers Adult Cat Food,3oz,N/A,Poultry Broth,5164703,None,NaN,http://www.petsmart.com/cat/food-and-health/fo...


In [5]:
#are there duplicated IDs? None in this test data set, but in the full data set there are.  
print products.duplicated('product_id').unique()

#which indices?
products[products.duplicated(['product_id'],keep=False)]

#Turns out all the duplicated product_id values in the full data set are nulls. 

[False]


,ash_max,crude_fat_min,crude_fiber_max,crude_protein_min,flavor,food_type,health_consideration,ingredients,life_stage,moisture_max,name,package_weight,price,primary_ingredient,product_id,tag,taurine_min,url


In [7]:
#to inspect the Beautiful Soup markup of a url that is a specific row of the dataframe:
link = products.iloc[3]['url']
r = urllib.urlopen(link)
soup_obj = BeautifulSoup(r, 'lxml') 
#print soup_obj

In [12]:
#inspecting one of the columns to see how clean the scrape was. The readout below is 
#from the full dataset. It did pretty well, though there are a couple instances
#of the wrong information, e.g.: '4 Chicken, 3 Chicken & Beef, and 3 Chicken & Duck.'. 

products.package_weight.unique()

array([u'3oz', u'5.5oz', nan, u'6 oz.', u'3 oz', u'2Lb', u'13oz',
       u'1.4 oz.', u'12Lb', u'5.5 oz', u'5Lb', u'22 lbs.', u'8 lbs.',
       u'15Lb', u'3 oz.', u'3.15Lb', u'5.5 oz.', u'4 lbs.', u'3Lb',
       u'16Lb', u'2.6 oz.', u'7Lb', u'(12) 2.75oz', u'2.8 oz.', u'6.5Lb',
       u'3.5Lb', u'11Lb', u'6Lb', u'22Lb', u'2.5 oz.', u'6.3Lb',
       u'11 lbs.', u'2.47 oz.', u'32ct', u'5 lbs.', u'14Lb', u'13.5oz',
       u'1.36 kg. (3 lbs.)', u'24ct of 5.5oz cans', u'5.8 oz.', u'15lb',
       u'4 Salmon, 3 Snapper & 3 Shrimp.', u'24ct', u'12 (3oz) Pouches',
       u'2.75 oz.', u'4.5LB', u'7 lbs.', u'12 Packs', u'24ct of 3oz',
       u'5.5 lbs.', u'12 Pack', u'2.5Lb', u'5 lb', u'11.5 lbs.',
       u'11.3 lbs.', u'6.75oz', u'13.5Lb', u'14.2Lb',
       u'(3)Tuna, (4)Tuna & Whitefish, (3)Sardine & Mackerel Entr\xc3\xa9es',
       u'5.1 lbs.', u'12ct', u'2 Lb',
       u'4 Chicken, 3 Chicken & Beef, and 3 Chicken & Duck.',
       u'5Lb\r\n14oz', u'1.98 lbs.', u'5.14Lb', u'5 oz.',
       u'12 pa